# 🧠 Módulo 5 — Programación Funcional en Python

Python no es 100% funcional, pero soporta muchas herramientas del paradigma:

- Funciones como ciudadanos de primera clase
- Funciones puras
- Lambdas
- `map`, `filter`, `reduce`
- Higher-order functions
- Closures

Este notebook explica cómo usarlas de forma realista en Python moderno.

---
## 1️⃣ Funciones como "ciudadanos de primera clase"

En Python, las funciones:
- se pueden almacenar en variables
- pasar como parámetros
- devolver desde otras funciones
- guardarse en estructuras


In [ ]:
def saludar():
    return "Hola"

f = saludar
f(), f()

---
## 2️⃣ Funciones puras

Una **función pura**:
- Siempre produce el mismo resultado con los mismos parámetros
    - Sin efectos secundarios
    - No modifica variables externas


In [ ]:
def pura(x, y):
    return x + y

pura(2,3), pura(2,3)

Una función **no pura** afecta o depende del estado externo:

In [ ]:
contador = 0
def impura(x):
    global contador
    contador += x
    return contador

impura(1), impura(1)

---
## 3️⃣ Lambdas

Funciones pequeñas y anónimas:

In [ ]:
cuadrado = lambda x: x*x
cuadrado(7)

---
## 4️⃣ `map`, `filter`, `reduce`

### ✔️ map: aplicar transformación

In [ ]:
nums = [1,2,3,4]
list(map(lambda x: x*2, nums))

### ✔️ filter: filtrar valores

In [ ]:
list(filter(lambda x: x%2==0, nums))

### ✔️ reduce: acumular resultados
Necesita import explícito:

In [ ]:
from functools import reduce
reduce(lambda acc, x: acc+x, nums)

---
## 5️⃣ Higher-Order Functions (HOF)

Una **HOF** es una función que:
- recibe otra función como argumento
- o devuelve una función nueva

Ejemplo: creador de multiplicadores:

In [ ]:
def multiplicador(n):
    def multiplicar(x):
        return x*n
    return multiplicar

por_3 = multiplicador(3)
por_3(10)

---
## 6️⃣ Closures

Una función interna recuerda las variables de su función externa, incluso después de haber terminado.

In [ ]:
def contador():
    n = 0
    def inc():
        nonlocal n
        n += 1
        return n
    return inc

c = contador()
c(), c(), c()

---
## 7️⃣ Ejemplo real: pipeline funcional

Pipeline para limpieza de datos:

In [ ]:
datos = ["  Hola  ", "MUNDO", "  Python "]
resultado = list(
    map(lambda t: t.strip().lower(), datos)
)
resultado

---
## 8️⃣ Ejercicio práctico

### �� Ejercicio
Dado:
```python
numeros = [1,2,3,4,5,6,7,8,9,10]
```
Usa **solo funciones funcionales** (`map`, `filter`, `reduce`) para obtener:

1. Los números pares
2. Su cuadrado
3. La suma final

Escribe tu solución aquí:

In [ ]:
# Escribe tu solución aquí


---
## ✅ Solución (oculta)

<details>
<summary>Mostrar solución</summary>

```python
from functools import reduce

pares = list(filter(lambda x: x%2==0, numeros))
cuadrados = list(map(lambda x: x*x, pares))
total = reduce(lambda acc,x: acc+x, cuadrados)

pares, cuadrados, total
```</details>